In [1]:
import sys
import os
import gc
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np

path_build = "../../data/tencent2020/build2/"
path_save = "../../data/tencent2020/save/"

pickle_path = "../../data/tencent2020/pickle"
if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
pd.set_option('display.max_columns', None)

In [2]:
click_log = pd.read_pickle(path_build + 'click_log.pkl')
train_user = pd.read_pickle(path_build + 'train_user.pkl')
print(click_log.shape, train_user.shape)

(133878445, 9) (3000000, 3)


In [3]:
click_log.sort_values(by=['user_id','time'], ascending=[True,True], inplace=True)
click_log = click_log.reset_index(drop=True)
click_log = click_log[click_log['click_times']<=8] #去除点击次数99.99%外的数据记录
print(click_log.shape)
click_log.head()

(133869604, 9)


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,1,821396,1,724607,10005,5,7293,326
1,20,1,209778,1,188507,136,2,9702,6
2,20,1,877468,1,773445,10005,5,29455,106
3,39,1,1683713,1,1458878,10005,5,14668,326
4,40,1,122032,1,109959,1334,2,11411,100


In [12]:
click_log['ad_id'].min(), click_log['ad_id'].max(), click_log['ad_id'].nunique()

(1, 3812202, 3812199)

In [4]:
click_log['creative_id_times'] = click_log['creative_id'].astype(np.int64)*10000000 + click_log['click_times']
click_log['ad_id_times'] = click_log['ad_id'].astype(np.int64)*10000000 + click_log['click_times']
click_log['product_id_times'] = click_log['product_id'].astype(np.int64)*10000000 + click_log['click_times']
click_log['advertiser_id_times'] = click_log['advertiser_id'].astype(np.int64)*10000000 + click_log['click_times']
click_log['product_category_times'] = click_log['product_category'].astype(np.int64)*10000000 + click_log['click_times']
click_log['industry_times'] = click_log['industry'].astype(np.int64)*10000000 + click_log['click_times']


# click_log['creative_id_times'] = click_log['creative_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['ad_id_times'] = click_log['ad_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['product_id_times'] = click_log['product_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['advertiser_id_times'] = click_log['advertiser_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['product_category_times'] = click_log['product_category'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['industry_times'] = click_log['industry'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,creative_id_times,ad_id_times,product_id_times,advertiser_id_times,product_category_times,industry_times
0,20,1,821396,1,724607,10005,5,7293,326,8213960000001,7246070000001,100050000001,72930000001,50000001,3260000001
1,20,1,209778,1,188507,136,2,9702,6,2097780000001,1885070000001,1360000001,97020000001,20000001,60000001
2,20,1,877468,1,773445,10005,5,29455,106,8774680000001,7734450000001,100050000001,294550000001,50000001,1060000001
3,39,1,1683713,1,1458878,10005,5,14668,326,16837130000001,14588780000001,100050000001,146680000001,50000001,3260000001
4,40,1,122032,1,109959,1334,2,11411,100,1220320000001,1099590000001,13340000001,114110000001,20000001,1000000001


In [5]:
click_log = multi_column_LabelEncoder(click_log, 
                                      columns=['creative_id_times','ad_id_times','product_id_times',
                                               'advertiser_id_times','product_category_times', 'industry_times',
                                              ])

print(click_log['creative_id'].nunique())
print(click_log['ad_id'].nunique())
print(click_log['product_id'].nunique())
print(click_log['product_category'].nunique())
print(click_log['advertiser_id'].nunique())
print(click_log['industry'].nunique())
print(click_log['creative_id_times'].nunique())
print(click_log['ad_id_times'].nunique())
print(click_log['product_id_times'].nunique())
print(click_log['advertiser_id_times'].nunique())
print(click_log['product_category_times'].nunique())
print(click_log['industry_times'].nunique())

creative_id_times LabelEncoder......
ad_id_times LabelEncoder......
product_id_times LabelEncoder......
advertiser_id_times LabelEncoder......
product_category_times LabelEncoder......
industry_times LabelEncoder......
LabelEncoder Successfully!
4445717
3812199
44313
18
62965
335
5487028
4816008
68162
120495
115
1798


In [6]:
from collections import defaultdict
import sys
creative_id = defaultdict(list)
ad_id = defaultdict(list)
product_id = defaultdict(list)
product_category = defaultdict(list)
advertiser_id = defaultdict(list)
industry = defaultdict(list)
click_times = defaultdict(list)
time = defaultdict(list)
creative_id_times = defaultdict(list)
ad_id_times = defaultdict(list)
product_id_times = defaultdict(list)
advertiser_id_times = defaultdict(list)
product_category_times = defaultdict(list)
industry_times = defaultdict(list)


for user,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14  in zip(click_log['user_id'], 
                            click_log['creative_id'],
                            click_log['ad_id'],
                            click_log['product_id'],
                            click_log['product_category'],
                            click_log['advertiser_id'],
                            click_log['industry'],
                            click_log['click_times'],
                            click_log['time'],
                            click_log['creative_id_times'],
                            click_log['ad_id_times'],
                            click_log['product_id_times'],
                            click_log['advertiser_id_times'],
                            click_log['product_category_times'],
                            click_log['industry_times']):
    creative_id[user].append(f1)
    ad_id[user].append(f2)
    product_id[user].append(f3)
    product_category[user].append(f4)
    advertiser_id[user].append(f5)
    industry[user].append(f6)
    click_times[user].append(f7)
    time[user].append(f8)
    
    creative_id_times[user].append(f9)
    ad_id_times[user].append(f10)
    product_id_times[user].append(f11)
    advertiser_id_times[user].append(f12)
    product_category_times[user].append(f13)
    industry_times[user].append(f14)
    
user_ids = pd.DataFrame({'user_id':list(creative_id.keys()),
                         'creative_id':list(creative_id.values()),
                        'ad_id':list(ad_id.values()),
                         'product_id':list(product_id.values()),
                         'product_category':list(product_category.values()),
                         'advertiser_id':list(advertiser_id.values()),
                         'industry':list(industry.values()),
                         'click_times':list(click_times.values()),
                         'time':list(time.values()),
                         
                         'creative_id_times':list(creative_id_times.values()),
                         'ad_id_times':list(ad_id_times.values()),
                         'product_id_times':list(product_id_times.values()),
                         'advertiser_id_times':list(advertiser_id_times.values()),
                         'product_category_times':list(product_category_times.values()),
                         'industry_times':list(industry_times.values()),
                        
                        
                        })
print(user_ids.shape)
user_ids.head()

(4000000, 15)


,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_times,ad_id_times,product_id_times,advertiser_id_times,product_category_times,industry_times
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...","[724607, 188507, 773445, 1458878, 109959, 6621...","[10005, 136, 10005, 10005, 1334, 10018, 10005,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[7293, 9702, 29455, 14668, 11411, 14681, 17189...","[326, 6, 106, 326, 100, 326, 73, 217, 64, 238,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[1063182, 293331, 1138617, 2130126, 176368, 10...","[958580, 269566, 1026121, 1889685, 162737, 997...","[20301, 490, 20301, 20301, 2679, 20352, 20301,...","[12541, 16889, 58163, 26349, 20377, 26387, 315...","[30, 7, 30, 30, 7, 108, 30, 30, 108, 7, 7, 7, 8]","[1732, 39, 526, 1732, 499, 1732, 380, 1102, 34..."
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...","[58788, 139702, 38066, 541125, 14495, 392680, ...","[87, 80, 129, 129, 1400, 87, 10018, 1261, 1001...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[22885, 10686, 18562, 25932, 768, 22885, 34505...","[318, 238, 6, 6, 317, 318, 47, 6, 47, 47, 242,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...","[93754, 222023, 58596, 790086, 18933, 581326, ...","[88571, 203960, 56789, 716490, 20602, 528123, ...","[277, 238, 459, 459, 2899, 277, 20352, 2444, 2...","[44043, 18905, 34552, 50889, 1333, 44043, 6847...","[7, 7, 7, 7, 7, 7, 108, 7, 108, 108, 7, 108, 7...","[1679, 1246, 39, 39, 1671, 1679, 262, 39, 262,..."
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...","[586668, 713448, 629278, 728308, 527601, 64310...","[36256, 40905, 1674, 35985, 1674, 10018, 10018...","[17, 17, 2, 17, 2, 18, 18, 2, 2, 2, 18, 12, 2,...","[32974, 9877, 18492, 14186, 17018, 9058, 8371,...","[100, 100, 322, 100, 322, 6, 54, 6, 322, 322, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12, 13, 14, 14, 14, 17, 19, 22, 31, 36, 37, 4...","[855771, 1045997, 918547, 1068492, 770408, 939...","[774925, 943201, 830445, 963534, 698956, 84889...","[59449, 64457, 3722, 59160, 3722, 20352, 20352...","[65335, 17287, 34379, 25498, 31243, 15690, 145...","[102, 102, 7, 102, 7, 108, 108, 7, 7, 7, 108, ...","[499, 499, 1711, 499, 1711, 39, 292, 39, 1711,..."
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...","[37966, 524312, 511235, 1638619, 1698206, 1953...","[1862, 10018, 2625, 38743, 41265, 37758, 39904...","[2, 18, 2, 17, 17, 4, 17, 17, 18, 2, 2, 2, 2, ...","[19451, 7976, 13084, 12130, 23664, 811, 10172,...","[238, 25, 248, 100, 100, 100, 100, 100, 88, 31...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[8, 15, 41, 44, 48, 48, 48, 48, 49, 52, 58, 58...","[58389, 765852, 747484, 2384416, 2469119, 2836...","[56622, 694770, 678514, 2113043, 2186873, 2505...","[4294, 20352, 7118, 62143, 64855, 61056, 63389...","[36447, 13800, 23470, 21647, 45737, 1414, 1784...","[7, 108, 7, 102, 102, 23, 102, 102, 108, 7, 7,...","[1246, 136, 1308, 499, 499, 499, 499, 499, 452..."
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...","[265971, 314795, 24966, 41148, 751113, 925792,...","[10005, 10008, 87, 136, 10018, 136, 10005, 100...","[5, 8, 2, 2, 18, 2, 5, 18, 18, 2, 18, 2, 5, 5,...","[11882, 992, 22885, 9706, 38760, 2862, 17745, ...","[297, 100, 318, 6, 322, 6, 288, 322, 319, 238,...","[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 13, 14, 15, 20, 21, 24, 25, 27, 28, 29, 30...","[401225, 469363, 35978, 63924, 1105070, 135513...","[367911, 429875, 36734, 61628, 995584, 1216710...","[20301, 20316, 277, 490, 20352, 491, 20301, 20...","[21154, 1925, 44043, 16910, 76947, 5468, 32796...","[30, 50, 7, 7, 108, 8, 30, 108, 108, 7, 108, 7...","[1562, 499, 1679, 39, 1711, 40, 1494, 1711, 16..."


In [7]:
del creative_id, ad_id, product_id, product_category, advertiser_id, industry,click_times,time, creative_id_times,
del ad_id_times,product_id_times,advertiser_id_times, product_category_times, industry_times
gc.collect()

20

In [8]:
save_pickle(user_ids, path_build + "user_ids.pkl")